# Preprocessing RoadTrafficAccidents

This file is for preprocessing the RoadTrafficAccidentsLocations file from https://data.stadt-zuerich.ch/dataset/sid_dav_strassenverkehrsunfallorte. 

In [1]:
import pandas as pd
import os
import datetime
import math
import numpy as np

Getting the full path to the dataset folder

In [2]:
dir_path = os.path.dirname(os.path.realpath("__file__"))
path_to_datasets_folder = os.path.join(dir_path, "datasets")
path_to_accidents_file = os.path.join(path_to_datasets_folder, "RoadTrafficAccidentLocations.csv")

Reading the TrafficAccident file and remove all language duplicates

In [3]:
df_accidents = pd.read_csv(path_to_accidents_file)
df_accidents = df_accidents.drop(columns=['AccidentType_de', 'AccidentType_fr', 'AccidentType_it'])
df_accidents = df_accidents.drop(columns=['AccidentSeverityCategory_de', 'AccidentSeverityCategory_fr', 'AccidentSeverityCategory_it'])
df_accidents = df_accidents.drop(columns=['RoadType_de', 'RoadType_fr', 'RoadType_it'])
df_accidents = df_accidents.drop(columns=['AccidentMonth_de', 'AccidentMonth_fr','AccidentMonth_it'])
df_accidents = df_accidents.drop(columns=['AccidentWeekDay_de', 'AccidentWeekDay_fr', 'AccidentWeekDay_it'])
df_accidents.head()

,AccidentUID,AccidentType,AccidentType_en,AccidentSeverityCategory,AccidentSeverityCategory_en,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,...,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_en,AccidentWeekDay,AccidentWeekDay_en,AccidentHour,AccidentHour_text
0,A2D2677533867004E0430A865E337004,at0,Accident with skidding or self-accident,as4,Accident with property damage,False,False,False,rt433,Minor road,...,1245194,ZH,261,2011,1,January,aw406,Saturday,0.0,00h-01h
1,9FD6441F802C20A6E0430A865E3320A6,at0,Accident with skidding or self-accident,as3,Accident with light injuries,False,True,False,rt433,Minor road,...,1246980,ZH,261,2011,1,January,aw406,Saturday,1.0,01h-02h
2,9FDA0DC4856A6094E0430A865E336094,at0,Accident with skidding or self-accident,as4,Accident with property damage,False,False,False,rt439,Other,...,1247749,ZH,261,2011,1,January,aw406,Saturday,2.0,02h-03h
3,A3B66E42396E6000E0430A865E336000,at5,Accident when crossing the lane(s),as3,Accident with light injuries,False,False,False,rt433,Minor road,...,1247102,ZH,261,2011,1,January,aw406,Saturday,2.0,02h-03h
4,9FDA0DBE8CCE9096E0430A865E339096,at0,Accident with skidding or self-accident,as4,Accident with property damage,False,False,False,rt433,Minor road,...,1250690,ZH,261,2011,1,January,aw406,Saturday,3.0,03h-04h


Create the correct datetime for each row and add 'nan' as value in the date row if the accident hour is missing (rows cases)

In [4]:
# Dims of cleaned dataset
df_accidents.head().shape

(5, 21)

In [5]:
from collections import Counter

# Count NaNs in dataset:
for i in df_accidents.columns:
    print('%s: ' %i, Counter(list(df_accidents[i].isna()))[1])

AccidentUID:  0
AccidentType:  0
AccidentType_en:  0
AccidentSeverityCategory:  0
AccidentSeverityCategory_en:  0
AccidentInvolvingPedestrian:  0
AccidentInvolvingBicycle:  0
AccidentInvolvingMotorcycle:  0
RoadType:  0
RoadType_en:  0
AccidentLocation_CHLV95_E:  0
AccidentLocation_CHLV95_N:  0
CantonCode:  0
MunicipalityCode:  0
AccidentYear:  0
AccidentMonth:  0
AccidentMonth_en:  0
AccidentWeekDay:  0
AccidentWeekDay_en:  0
AccidentHour:  4
AccidentHour_text:  0


In [6]:
# Checking positions of NaNs in dataset and get corresponding columns:
idx, idy = np.where(pd.isnull(df_accidents))
idx, df_accidents.columns[idy]

(array([3149, 5749, 7436, 8193], dtype=int64),
 Index(['AccidentHour', 'AccidentHour', 'AccidentHour', 'AccidentHour'], dtype='object'))

In [7]:
# Drop obs from 2020 since weather dataset only contains missing values for this year
df_accidents = df_accidents[df_accidents['AccidentYear'] != 2020]

Count days in every year's month... This is needed to check whether we have data for every day of each year in order to pin down the exact date:


In [8]:
from itertools import groupby
import calendar

years = np.arange(2011,2020,1).tolist()
months = np.arange(1,13,1).tolist()


# Count number of distinct days in each month for every year:
for j in years:
    
    days_per_month = []
    
    for i in months:
        z = pd.Series(zip([[len([v]), k] for k, v in groupby(df_accidents[(df_accidents['AccidentYear'] == j) & (df_accidents['AccidentMonth'] == i)]['AccidentWeekDay'])]))
        item_counts = z.index.value_counts(normalize=False)
        days_per_month.append(item_counts.sum())
        
    print('{}:'.format(j), days_per_month)

2011: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2012: [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2013: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2014: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2015: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2016: [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2017: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2018: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
2019: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


In [9]:
# Counting entries in every year's months:

for j in years:
    
    dict_ = {}
    
    for i in months:
        row_dim = df_accidents[(df_accidents['AccidentYear'] == j) & (df_accidents['AccidentMonth'] == i)].shape[0]
        months_name = calendar.month_name[i]
        dict_[months_name[:3]] = row_dim
    
    print('{}:'.format(j), dict_)
    print('')

2011: {'Jan': 274, 'Feb': 221, 'Mar': 288, 'Apr': 324, 'May': 360, 'Jun': 310, 'Jul': 290, 'Aug': 343, 'Sep': 332, 'Oct': 324, 'Nov': 305, 'Dec': 319}

2012: {'Jan': 269, 'Feb': 239, 'Mar': 340, 'Apr': 290, 'May': 306, 'Jun': 309, 'Jul': 307, 'Aug': 305, 'Sep': 280, 'Oct': 289, 'Nov': 322, 'Dec': 279}

2013: {'Jan': 242, 'Feb': 227, 'Mar': 299, 'Apr': 305, 'May': 332, 'Jun': 372, 'Jul': 337, 'Aug': 323, 'Sep': 299, 'Oct': 328, 'Nov': 314, 'Dec': 267}

2014: {'Jan': 247, 'Feb': 260, 'Mar': 321, 'Apr': 301, 'May': 311, 'Jun': 388, 'Jul': 294, 'Aug': 299, 'Sep': 301, 'Oct': 313, 'Nov': 252, 'Dec': 263}

2015: {'Jan': 249, 'Feb': 246, 'Mar': 298, 'Apr': 288, 'May': 324, 'Jun': 356, 'Jul': 550, 'Aug': 491, 'Sep': 512, 'Oct': 511, 'Nov': 519, 'Dec': 447}

2016: {'Jan': 450, 'Feb': 396, 'Mar': 420, 'Apr': 514, 'May': 549, 'Jun': 554, 'Jul': 478, 'Aug': 504, 'Sep': 546, 'Oct': 515, 'Nov': 480, 'Dec': 412}

2017: {'Jan': 411, 'Feb': 398, 'Mar': 543, 'Apr': 457, 'May': 547, 'Jun': 536, 'Jul': 49

Safe null time values as NaN and figure out each date and save as datetime

In [10]:
first_day = 0
day_multiplier = 0
start = 0
prev_days = ('aw402', 'aw403', 'aw404', 'aw405', 'aw406', 'aw407')

for i, row in df_accidents.iterrows():
    
    # Set the reducer variable according to which day the current month starts with
    # This is done so that in the if, elif loops below to get the correct day for
    # every month
    
    reducer = datetime.date(row['AccidentYear'], row['AccidentMonth'], 1).weekday()
    
    # increase the day multiplier if the a "higher" Weekday is followed by a "lower" weekday
    # example Saturday is followed by a thursday, in this case it is inferred that the next 
    # week has started
    
    if start > 0:
        if int(df_accidents.at[i-1, 'AccidentWeekDay'][-1]) > int(df_accidents.at[i, 'AccidentWeekDay'][-1]):
            day_multiplier += 1
            
        # Reset the day_multiplier if a new month starts
        
        if df_accidents.at[i-1, 'AccidentMonth'] != df_accidents.at[i, 'AccidentMonth']:
            day_multiplier = 0
        
        
    
    if row['AccidentWeekDay'] == 'aw401':
        df_accidents.at[i, 'day'] = 1 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw402':
        df_accidents.at[i, 'day'] = 2 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw403':
        df_accidents.at[i, 'day'] = 3 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw404':
        df_accidents.at[i, 'day'] = 4 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw405':
        df_accidents.at[i, 'day'] = 5 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw406':
        df_accidents.at[i, 'day'] = 6 + 7 * day_multiplier - reducer
    elif row['AccidentWeekDay'] == 'aw407':
        df_accidents.at[i, 'day'] = 7 + 7 * day_multiplier - reducer
    
    # Safe null values in Accident hour as 'nan' in date column
    if math.isnan(df_accidents.at[i, 'AccidentHour']):
        accident_date = df_accidents.at[i, 'AccidentHour']
        print(df_accidents.at[i, 'AccidentHour'])
        
    # Convert to datetime format all time with :30
    else:
        accident_date = datetime.datetime(row['AccidentYear'], row['AccidentMonth'], int(df_accidents.at[i, 'day']), int(df_accidents.at[i, 'AccidentHour']), 30)

    df_accidents.at[i, 'date'] = accident_date
    
    start = 1

nan
nan
nan
nan


In [11]:
# Drop columns in accidents dataframe that are not necessary
df_accidents = df_accidents.drop(['AccidentUID', 'day', 'AccidentHour_text', 'AccidentWeekDay'], axis = 1)
df_accidents = df_accidents.drop(['AccidentType_en', 'AccidentSeverityCategory_en', 'RoadType_en', 'AccidentSeverityCategory_en',
                                 'CantonCode', 'MunicipalityCode', 'AccidentMonth_en'], axis = 1)

In [12]:
# Convert boolean cols to boolean values:
df_accidents['AccidentInvolvingPedestrian'] = 1*df_accidents['AccidentInvolvingPedestrian'].values
df_accidents['AccidentInvolvingBicycle'] = 1*df_accidents['AccidentInvolvingBicycle'].values
df_accidents['AccidentInvolvingMotorcycle'] = 1*df_accidents['AccidentInvolvingMotorcycle'].values

In [13]:
# Move date column in accidents df to first position
first_column = df_accidents.pop('date')
df_accidents.insert(0, 'date', first_column)

#### Rename columns of month, hour and weekday for merge later

In [14]:
month_columns = [df_accidents[column].name for column in df_accidents if df_accidents[column].name.startswith('AccidentMonth')]
hour_columns = [df_accidents[column].name for column in df_accidents if df_accidents[column].name.startswith('AccidentHour')]
year_columns = [df_accidents[column].name for column in df_accidents if df_accidents[column].name.startswith('AccidentYear')]


for i in month_columns:
    df_accidents.rename({i: i[8:]}, axis = 1, inplace= True)
    
for i in hour_columns:
    df_accidents.rename({i: i[8:]}, axis = 1, inplace = True)
    
for i in year_columns:
    df_accidents.rename({i: i[8:]}, axis = 1, inplace = True)
    
    
df_accidents.rename({'AccidentWeekDay_en': 'WeekDay'}, axis = 1, inplace = True)

df_accidents.columns

Index(['date', 'AccidentType', 'AccidentSeverityCategory',
       'AccidentInvolvingPedestrian', 'AccidentInvolvingBicycle',
       'AccidentInvolvingMotorcycle', 'RoadType', 'AccidentLocation_CHLV95_E',
       'AccidentLocation_CHLV95_N', 'Year', 'Month', 'WeekDay', 'Hour'],
      dtype='object')

In [15]:
df_accidents.head()

,date,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,Year,Month,WeekDay,Hour
0,2011-01-01 00:30:00,at0,as4,0,0,0,rt433,2684605,1245194,2011,1,Saturday,0.0
1,2011-01-01 01:30:00,at0,as3,0,1,0,rt433,2682382,1246980,2011,1,Saturday,1.0
2,2011-01-01 02:30:00,at0,as4,0,0,0,rt439,2682791,1247749,2011,1,Saturday,2.0
3,2011-01-01 02:30:00,at5,as3,0,0,0,rt433,2681199,1247102,2011,1,Saturday,2.0
4,2011-01-01 03:30:00,at0,as4,0,0,0,rt433,2682479,1250690,2011,1,Saturday,3.0


In [16]:
# Check for NaNs:
df_accidents.isnull().any()

date                            True
AccidentType                   False
AccidentSeverityCategory       False
AccidentInvolvingPedestrian    False
AccidentInvolvingBicycle       False
AccidentInvolvingMotorcycle    False
RoadType                       False
AccidentLocation_CHLV95_E      False
AccidentLocation_CHLV95_N      False
Year                           False
Month                          False
WeekDay                        False
Hour                            True
dtype: bool

In [17]:
df_accidents[df_accidents['date'].isnull()]

,date,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,Year,Month,WeekDay,Hour
3149,NaT,at0,as4,0,0,0,rt430,2681888,1246482,2011,11,Monday,NaN
5749,NaT,at0,as4,0,0,0,rt430,2679335,1249796,2012,7,Tuesday,NaN
7436,NaT,at0,as4,0,0,0,rt431,2683660,1250338,2013,1,Monday,NaN
8193,NaT,at0,as4,0,0,0,rt430,2684363,1250915,2013,4,Friday,NaN


In [18]:
# Drop NaNs:
df_accidents = df_accidents.dropna()
df_accidents

,date,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,Year,Month,WeekDay,Hour
0,2011-01-01 00:30:00,at0,as4,0,0,0,rt433,2684605,1245194,2011,1,Saturday,0.0
1,2011-01-01 01:30:00,at0,as3,0,1,0,rt433,2682382,1246980,2011,1,Saturday,1.0
2,2011-01-01 02:30:00,at0,as4,0,0,0,rt439,2682791,1247749,2011,1,Saturday,2.0
3,2011-01-01 02:30:00,at5,as3,0,0,0,rt433,2681199,1247102,2011,1,Saturday,2.0
4,2011-01-01 03:30:00,at0,as4,0,0,0,rt433,2682479,1250690,2011,1,Saturday,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42725,2019-12-31 12:30:00,at0,as4,0,1,1,rt433,2683523,1247624,2019,12,Tuesday,12.0
42726,2019-12-31 18:30:00,at5,as3,0,1,0,rt433,2681573,1247700,2019,12,Tuesday,18.0
42727,2019-12-31 18:30:00,at1,as4,0,0,0,rt432,2682276,1247052,2019,12,Tuesday,18.0
42728,2019-12-31 19:30:00,at7,as4,0,0,0,rt433,2683004,1247184,2019,12,Tuesday,19.0


In [19]:
df_accidents.to_pickle("datasets\Accidents.pkl")